In [1]:
import fix_sys_path
from plasma_classes import *
from plasma_utils import *
from matplotlib import pyplot as plt
from matplotlib import animation
from tqdm import tqdm
import os
from tempfile import TemporaryDirectory

In [7]:
def loop_over_states(file_path: str):
    """
    A generator that yields the system's state at each iteration from a binary file.

    file_path: str - the path to the binary file to read from.
    Yields:
        - A tuple containing the nodes object, the particles object, and the walls object.
    """
    # Open the file in read-only mode.
    with open(file_path, "rb") as f:
        # Compute the position in the file where the serialized data begins.
        header_size = 4  # The size of the header containing the size of the serialized data.
        data_offset = 0

        while True:
            # Move the file pointer to the beginning of the serialized data.
            f.seek(data_offset, os.SEEK_SET)

            # Read the size of the serialized data.
            size_bytes = f.read(header_size)
            if not size_bytes:
                # No more data in the file.
                break
            size = int.from_bytes(size_bytes, byteorder="big")

            # Compute the position in the file where the next serialized data begins.
            data_offset = f.tell() + size

            # Read the serialized data.
            serialized_bytes = f.read(size)
            serialized_data = pickle.loads(serialized_bytes)

            # Yield the iteration number, nodes, particles, and walls from the deserialized data.
            yield serialized_data["nodes"], serialized_data["particles"], serialized_data["walls"]


In [104]:
import copy

def deepcopy_nodes(nodes):
    new_nodes = Nodes(nodes.length - 1)
    new_nodes.rho = np.copy(nodes.rho)
    new_nodes.E = np.copy(nodes.E)
    new_nodes.phi = np.copy(nodes.phi)
    new_nodes.electricEnergy = np.copy(nodes.electricEnergy)
    new_nodes.conc_e = np.copy(nodes.conc_e)
    new_nodes.conc_i = np.copy(nodes.conc_i)
    return new_nodes

def deepcopy_wall(wall):
    new_wall = Wall(wall.left, wall.right, wall.number, wall.h, wall.side)
    new_wall.particles_lst = copy.deepcopy(wall.particles_lst)
    return new_wall


In [105]:
# Create a temporary directory to store the test files.
# Set up the test data.
n_macro = 10
concentration = 1e15
q = 1.602176634e-19
m = 9.10938356e-31
nodes = Nodes(10)
particles = Particles(n_macro, concentration, -q, m)
walls = [Wall(0, 0.1, 1, 1e-3, "left"), Wall(0.9, 1, 2, 1e-3, "right")]

# Create a list to store the system's state for each iteration.
system_states = []

# Modify the system's state for 5 iterations and save the system's state after each iteration.
for i in range(5):
    # Modify the system's state.
    nodes.rho += 10
    particles.v += 1
    walls[0].particles_lst = [particles]

    # Save the system's state to a file.
    file_path = "test.bin"
    save_system_state(i, nodes, particles, walls, file_path)

    # Append the system's state to the list.
    system_states.append((deepcopy_nodes(nodes), particles.deepcopy(), deepcopy_wall(walls[0])))

# Load the system's state for the first and third iterations.
nodes_0, particles_0, walls_0 = load_system_state("test.bin", 0)
nodes_2, particles_2, walls_2 = load_system_state("test.bin", 2)

# Check that the loaded data matches the expected values.
assert np.array_equal(nodes_0.rho, system_states[0][0].rho)
assert np.array_equal(particles_0.v, system_states[0][1].v)
assert np.array_equal(walls_0.particles_lst, system_states[0][2].particles_lst)

assert np.array_equal(nodes_2.rho, system_states[2][0].rho)
assert np.array_equal(particles_2.v, system_states[2][1].v)
assert np.array_equal(walls_2.particles_lst, system_states[2][2].particles_lst)


# Load the system's state for a non-existent iteration and check that an exception is raised.
try:
    load_system_state(os.path.join("test.bin"), 5)
    assert False, "Expected ValueError not raised."
except ValueError as e:
    assert str(e) == "No data for iteration 5 found in file test_0.bin"


----
-1
----
-1
0
1


AttributeError: 'list' object has no attribute 'particles_lst'

In [115]:
walls_0[0].particles_lst[0].v

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [108]:
particles_0.v

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [109]:
system_states[0][0].rho

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.])

In [110]:
nodes_0.rho

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.])

In [11]:
p1 = Particles(10, 1.0, -1.0, 1.0)
p1.x = np.arange(10)
p1.v = np.arange(10)
p2 = Particles(10, 1.0, -1.0, 1.0)
p2.x = np.arange(10)
p2.v = np.arange(10)

In [12]:
def bar(first, second):
    first.add(second)

In [13]:
bar(p1, p2)

In [14]:
p1.x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
particles.x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
def foo(particles):
    mask = particles.x < 4
    particles.delete(mask)

In [4]:
foo(particles)

In [5]:
particles.x

array([4, 5, 6, 7, 8, 9])

In [15]:
mask = particles.x < 4

In [16]:
mask

array([ True,  True,  True,  True, False, False, False, False, False,
       False])

In [17]:
particles = particles[mask]

In [18]:
particles.x

array([0, 1, 2, 3])

In [2]:
import time

# Create 10 million particles
particles = Particles(10000000, 1.0, -1.0, 1.0)

# Modify individual particle velocities one at a time
start_time = time.time()
for i in range(5000000):
    particles.v[i] += 0.1
end_time = time.time()

print("Elapsed time: ", end_time - start_time, " seconds")

Elapsed time:  0.5579440593719482  seconds


In [3]:
slice1 = particles[3:5000000]

In [4]:
# Modify individual particle velocities one at a time
start_time = time.time()
for i in range(3000):
    slice1.v[i] += 1000
end_time = time.time()

print("Elapsed time: ", end_time - start_time, " seconds")

Elapsed time:  0.0  seconds


In [5]:
import numpy as np
import time

# Set up the particles array
n_particles = 100000
x = np.random.uniform(low=-1, high=1, size=n_particles)
v = np.random.normal(loc=0, scale=1, size=n_particles)
particles = Particles(n_particles, 1.0, 1.0, 1.0)
particles.x = x
particles.v = v

# Create a boolean mask
mask = np.random.choice([True, False], size=n_particles)

# Test the boolean index operation
t1 = time.time()
sub_particles = particles[mask]
t2 = time.time()
print(f"Boolean index operation took {t2-t1} seconds")

# Test the by-hand creation of new array with the same mask
t3 = time.time()
sub_particles = Particles(np.count_nonzero(mask), particles.concentration, particles.q, particles.m)
sub_particles.x = particles.x[mask]
sub_particles.v = particles.v[mask]
t4 = time.time()
print(f"By-hand creation of new array with the same mask took {t4-t3} seconds")


Boolean index operation took 0.0009968280792236328 seconds
By-hand creation of new array with the same mask took 0.0009965896606445312 seconds


In [6]:
# create original particles object
original_particles = Particles(10, 1.0, 1.0, 1.0)
original_particles.x = np.random.rand(10)

# make a deep copy of the original particles object
copy_particles = original_particles.deepcopy()

# modify the copy
copy_particles.x[0] = 0.0

# check that the original particles object is not modified
assert np.allclose(original_particles.x[0], 0.0) == False


In [7]:
original_particles.x

array([0.92823824, 0.76024596, 0.53362671, 0.0998817 , 0.07057969,
       0.72800722, 0.4734317 , 0.77427962, 0.16064182, 0.1847532 ])

In [8]:
sl = original_particles[2:7].deepcopy()
sl.x *= 10000
sl.x

array([5336.26709348,  998.8169768 ,  705.79688562, 7280.07222921,
       4734.31700472])

In [9]:
original_particles.x

array([0.92823824, 0.76024596, 0.53362671, 0.0998817 , 0.07057969,
       0.72800722, 0.4734317 , 0.77427962, 0.16064182, 0.1847532 ])

In [7]:
particles.v[3:]

array([2000.1, 2000.1, 2000.1, ...,    0. ,    0. ,    0. ])

In [42]:
slice6 = particles[3456 :5000000]

In [43]:
# Modify individual particle velocities one at a time
start_time = time.time()
for i in range(5000000):
    slice1.v[i] -= 2000
end_time = time.time()

print("Elapsed time: ", end_time - start_time, " seconds")

Elapsed time:  0.6053729057312012  seconds


In [44]:
particles.v

array([-999.9, -999.9, -999.9, ...,    0. ,    0. ,    0. ])

In [45]:
slc = particles[:20]

In [46]:
slc.x

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [48]:
particles.x[:20] = []
slc.x

ValueError: could not broadcast input array from shape (0,) into shape (20,)

In [29]:
# Create a Particles object with 100 particles
particles = Particles(100, 1.0, -1.0, 1.0)

# Select a slice of the particles
sliced_particles = particles[10:30]

# Double the speed of the sliced particles
sliced_particles.v *= 2.0


# Verify that the parent Particles object was updated
assert np.allclose(particles.v[10:30], sliced_particles.v)


In [3]:
import numpy as np
import os

# Create instances of each class for testing
particles = Particles(100, 1.0, 1.0, 1.0)
particles.x = np.random.rand(100)
particles.v = np.random.rand(100)
nodes = Nodes(10)
nodes.rho = np.random.rand(11)
nodes.E = np.random.rand(11)
nodes.phi = np.random.rand(11)
wall = Wall(0, 1, 10, 0.1, 'left')
particle1 = Particles(10, 0.5, 1.5, 1.0)
particle1.x = np.random.rand(10)
particle1.v = np.random.rand(10)
particle2 = Particles(5, 0.5, 1.5, 1.0)
particle2.x = np.random.rand(5)
particle2.v = np.random.rand(5)
wall.particles_lst = [particle1, particle2]

# Test saving and loading of Particles objects
particles_path = 'particles.pkl'
save_to_file(particles, particles_path)
loaded_particles = load_from_file(particles_path)
assert isinstance(loaded_particles, Particles)
assert loaded_particles.n_macro == particles.n_macro
assert loaded_particles.concentration == particles.concentration
assert np.allclose(loaded_particles.x, particles.x)
assert np.allclose(loaded_particles.v, particles.v)

# Test saving and loading of Nodes objects
nodes_path = 'nodes.pkl'
save_to_file(nodes, nodes_path)
loaded_nodes = load_from_file(nodes_path)
assert isinstance(loaded_nodes, Nodes)
assert loaded_nodes.length == nodes.length
assert np.allclose(loaded_nodes.rho, nodes.rho)
assert np.allclose(loaded_nodes.E, nodes.E)
assert np.allclose(loaded_nodes.phi, nodes.phi)

# Test saving and loading of Wall objects
wall_path = 'wall.pkl'
save_to_file(wall, wall_path)
loaded_wall = load_from_file(wall_path)
assert isinstance(loaded_wall, Wall)
assert loaded_wall.left == wall.left
assert loaded_wall.right == wall.right
assert loaded_wall.number == wall.number
assert loaded_wall.h == wall.h
assert loaded_wall.side == wall.side
assert len(loaded_wall.particles_lst) == len(wall.particles_lst)
assert isinstance(loaded_wall.particles_lst[0], Particles)
assert loaded_wall.particles_lst[0].n_macro == particle1.n_macro
assert loaded_wall.particles_lst[0].concentration == particle1.concentration
assert loaded_wall.particles_lst[0].q == particle1.q
assert loaded_wall.particles_lst[0].m == particle1.m
assert np.allclose(loaded_wall.particles_lst[0].x, particle1.x)
assert np.allclose(loaded_wall.particles_lst[0].v, particle1.v)
assert isinstance(loaded_wall.particles_lst[1], Particles)
assert loaded_wall.particles_lst[1].n_macro == particle2.n_macro
assert loaded_wall.particles_lst[1].concentration == particle2.concentration
assert loaded_wall.particles_lst[1].q == particle2.q
assert loaded_wall.particles_lst[1].m == particle2.m
assert np.allclose(loaded_wall.particles_lst[1].x, particle2.x)
assert np.allclose(loaded_wall.particles_lst[1].v, particle2.v)

# Clean up the test files
os.remove(particles_path)
os.remove(nodes_path)
os.remove(wall_path)
